In [1]:
import sys
sys.path.append('../../')
from utils.packages import *
from utils.ml_fairness import *
from utils.standard_data import *
dir = 'res/adult/'
Path(dir).mkdir(parents=True, exist_ok=True)

d_fields = ['Stage', 'CVR', 'CVD', 'V_SPD', 'V_EOD', 'V_AOD', 'V_ERD', 'Acc', 'F1','SPD', 'EOD', 'AOD', 'ERD']
diff_file = dir + 'diff' + '.csv'
if(not os.path.isfile(diff_file)):
    with open(diff_file, 'a') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(d_fields)
    
f_count = len([name for name in os.listdir(dir) if os.path.isfile(os.path.join(dir, name)) and not name.startswith('.')])
# fields = ['Acc', 'F1', 'DI','SPD', 'EOD', 'AOD', 'ERD', 'CNT', 'TI']
fields = ['Acc', 'F1', 'SPD', 'EOD', 'AOD', 'ERD']

filename = dir + 'glob-' + str(f_count) + '.csv'
with open(filename, 'a') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    

In [2]:
train_path = '../../data/adult/adult.data'
test_path = '../../data/adult/adult.test'
column_names = ['age', 'workclass', 'fnlwgt', 'education',
            'education-num', 'marital-status', 'occupation', 'relationship',
            'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
            'native-country', 'income-per-year']
na_values=['?']
train = pd.read_csv(train_path, header=None, names=column_names, skipinitialspace=True, na_values=na_values)
test = pd.read_csv(test_path, header=0, names=column_names, skipinitialspace=True, na_values=na_values)
df = pd.concat([test, train], ignore_index=True)

##### Drop na values
dropped = df.dropna()
count = df.shape[0] - dropped.shape[0]
print("Missing Data: {} rows removed.".format(count))
df = dropped

## Feature selection
# features_to_keep = ['workclass', 'education-num', 'marital-status', 'race', 'sex', 'relationship', 'capital-gain', 'capital-loss', 'income-per-year']
# # cat_feat = ['sex', 'workclass', 'marital-status', 'relationship']
# df = df[features_to_keep]

# Create a one-hot encoding of the categorical variables.
cat_feat = ['sex', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country']
df = pd.get_dummies(df, columns=cat_feat, prefix_sep='=')

# for feature in cat_feat:
#     le = LabelEncoder()
#     df[feature] = le.fit_transform(df[feature])


Missing Data: 3620 rows removed.


In [3]:
seed = 42 # randrange(100)
y2_train, y2_test = train_test_split(df, test_size = 0.3, random_state = seed) # stratify=df['race']
y1_train, y1_test = train_test_split(df, test_size = 0.3, random_state = seed) # 

In [4]:
### Imputation

# #### Drop na values
# y1_train = y1_train.dropna()
# y1_test = y1_test.dropna()
# y2_test = y1_test.dropna()

# le = LabelEncoder()
# for feature in cat_feat:
#     y1_train[feature] = le.fit_transform(y1_train[feature])
#     y2_test[feature] = le.fit_transform(y2_test[feature])
#     y1_test[feature] = le.fit_transform(y1_test[feature])

# from sklearn.impute import KNNImputer, MissingIndicator
# # imputer = SimpleImputer(missing_values=np.NaN, strategy='most_frequent')
# imputer = SimpleImputer(missing_values=np.NaN, strategy='constant', fill_value='unknown')
# # imputer = IterativeImputer(missing_values=np.NaN, initial_strategy='most_frequent')
# # imputer = KNNImputer(missing_values=np.NaN)
# # imputer = MissingIndicator(missing_values=np.NaN, strategy='most_frequent')
# y2_train_imputed = pd.DataFrame(imputer.fit_transform(y2_train))
# y2_train_imputed.columns = y2_train.columns
# y2_train_imputed.index = y2_train.index
# y2_train = y2_train_imputed

# for feature in cat_feat:
#     y2_train[feature] = le.fit_transform(y2_train[feature])

In [5]:
pro_att_name = ['race'] # ['race', 'sex']
priv_class = ['White'] # ['White', 'Male']
reamining_cat_feat = []

y2_data_orig_train, y2_X_train, y2_y_train = load_adult_data(y2_train, pro_att_name, priv_class, reamining_cat_feat)
y2_data_orig_test, y2_X_test, y2_y_test = load_adult_data(y2_test, pro_att_name, priv_class, reamining_cat_feat)

y1_data_orig_train, y1_X_train, y1_y_train = load_adult_data(y1_train, pro_att_name, priv_class, reamining_cat_feat)
y1_data_orig_test, y1_X_test, y1_y_test = load_adult_data(y1_test, pro_att_name, priv_class, reamining_cat_feat)


In [6]:
# from imblearn.pipeline import Pipeline as Pipe
# from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler, NearMiss, AllKNN
# from imblearn.combine import SMOTEENN

# over = SVMSMOTE(sampling_strategy='auto')
# under = AllKNN(sampling_strategy='auto')
# combine = SMOTEENN(sampling_strategy='auto')
# step_over = [('o', over)] # , ('u', under)
# step_under = [('u', under)] # , ('u', under)
# step_combined = [('c', combine)] # , ('u', under)
# # p = Pipe(steps=under)
# y2_X_train, y2_y_train = combine.fit_resample(y2_X_train, y2_y_train)

In [7]:
# sc = StandardScaler()
# sc = MinMaxScaler()
# sc = MaxAbsScaler()
# sc = RobustScaler()
sc = Normalizer(norm='l1')

from sklearn.preprocessing import QuantileTransformer, PowerTransformer
# sc = QuantileTransformer()
# sc = PowerTransformer()

y2_X_train = sc.fit_transform(y2_X_train)
y2_X_test = sc.fit_transform(y2_X_test)
y2_data_orig_train.features = y2_X_train
y2_data_orig_test.features = y2_X_test

In [8]:
from sklearn.feature_selection import SelectFpr, SelectPercentile, VarianceThreshold
# pca = PCA(n_components=5)
# pca = SparsePCA(n_components=5)
# pca = KernelPCA(n_components=5)

# pca = SelectKBest(k=5)
# pca = SelectFpr()
# pca = SelectPercentile()

# trained = pca.fit(y2_X_train, y2_y_train)
# y2_X_train = trained.transform(y2_X_train)
# y2_X_test = trained.transform(y2_X_test)
# y2_data_orig_test.features = y2_X_test


In [9]:
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier

y2_lr = LogisticRegression()
y1_lr = LogisticRegression()

y2_gbc = GradientBoostingClassifier()
y1_gbc = GradientBoostingClassifier()

y2_cbc = CatBoostClassifier()
y1_cbc = CatBoostClassifier()

y2_xgb = XGBClassifier()
y1_xgb = XGBClassifier()

y2_dct = DecisionTreeClassifier()
y1_dct = DecisionTreeClassifier()

classifiers = [(y1_lr, y2_lr), (y1_gbc, y2_gbc), (y1_cbc, y2_cbc), (y1_xgb, y2_xgb), (y1_dct, y2_dct)]

In [10]:
for clf in classifiers:
    y1_clf = clf[0]
    y2_clf = clf[1]
    
    y2_mdl = y2_clf.fit(y2_X_train, y2_y_train)
    y1_mdl = y1_clf.fit(y1_X_train, y1_y_train)

    # plot_model_performance(y2_mdl, y2_X_test, y2_y_test)
    y1_pred, y1_fair = get_fair_metrics_and_plot(filename, y1_data_orig_test, y1_mdl)
    y2_pred, y2_fair = get_fair_metrics_and_plot(filename, y2_data_orig_test, y2_mdl)

    y1_fair = y1_fair.drop(['DI', 'CNT', 'TI'], axis=1)
    y2_fair = y2_fair.drop(['DI', 'CNT', 'TI'], axis=1)
    
    CVR, CVD, AVR_EOD, AVD_EOD, AVR_SPD, AVD_SPD, AVD_AOD, AV_ERD = compute_new_metrics(y1_data_orig_test, y1_pred, y2_pred)
    row_y1 = y1_fair.iloc[[0]].values[0].tolist()
    row_y2 = y2_fair.iloc[[0]].values[0].tolist()
    diff = []
    
    with open(filename, 'a') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(row_y1)
        csvwriter.writerow(row_y2) 

    diff.append(CVR)
    diff.append(CVD)
    diff.append(AVD_SPD)
    diff.append(AVD_EOD)
    diff.append(AVD_AOD)
    diff.append(AV_ERD)

    for i in range(len(row_y2)):
        if(i < 2):
            change = row_y2[i] - row_y1[i]
        else:
            sign = ''
            if(row_y2[i] >= 0 and row_y1[i] >= 0):
                sign = '(+)'
                d = abs(row_y2[i]) - abs(row_y1[i])
            if(row_y2[i] < 0 and row_y1[i] < 0):
                sign = '(-)'
                d = abs(row_y2[i]) - abs(row_y1[i])
            if(row_y2[i] < 0 and row_y1[i] >= 0):
                sign = '(+-)'
                d = row_y2[i] - row_y1[i]
            if(row_y2[i] >=0 and row_y1[i] < 0):
                sign = '(-+)'
                d = row_y2[i] - row_y1[i]
            d = round(d, 3)
            change = sign + ' ' + str(d)

        diff.append(change)
 
    cols = ['CVR', 'CVD', 'AV_SPD', 'AV_EOD', 'AV_AOD', 'AV_ERD', 'Acc', 'F1','SPD', 'EOD', 'AOD', 'ERD']
    # metrics = pd.DataFrame(data=obj_fairness, index=['y1'], columns=cols)
    diff_df = pd.DataFrame(data=[diff], columns  = cols, index = ['Diff']).round(3)
    stage = 'Normalizer'
    diff = diff_df.iloc[0].values.tolist()
    diff.insert(0, stage)
      
    with open(diff_file, 'a') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(diff)    

Unprinv: race 0.0
Unprinv: race 0.0
Learning rate set to 0.045043
0:	learn: 0.6586823	total: 66.3ms	remaining: 1m 6s
1:	learn: 0.6286607	total: 80.1ms	remaining: 39.9s
2:	learn: 0.6005026	total: 93.1ms	remaining: 30.9s
3:	learn: 0.5741282	total: 114ms	remaining: 28.3s
4:	learn: 0.5519753	total: 131ms	remaining: 26.2s
5:	learn: 0.5326858	total: 150ms	remaining: 24.8s
6:	learn: 0.5139652	total: 165ms	remaining: 23.4s
7:	learn: 0.5001383	total: 177ms	remaining: 21.9s
8:	learn: 0.4856526	total: 189ms	remaining: 20.9s
9:	learn: 0.4742298	total: 201ms	remaining: 19.9s
10:	learn: 0.4629682	total: 214ms	remaining: 19.3s
11:	learn: 0.4544447	total: 226ms	remaining: 18.6s
12:	learn: 0.4473533	total: 239ms	remaining: 18.1s
13:	learn: 0.4401725	total: 252ms	remaining: 17.7s
14:	learn: 0.4326657	total: 263ms	remaining: 17.3s
15:	learn: 0.4260354	total: 276ms	remaining: 16.9s
16:	learn: 0.4198106	total: 288ms	remaining: 16.7s
17:	learn: 0.4143316	total: 300ms	remaining: 16.4s
18:	learn: 0.4100235	to

173:	learn: 0.3248571	total: 2.38s	remaining: 11.3s
174:	learn: 0.3247307	total: 2.39s	remaining: 11.3s
175:	learn: 0.3245885	total: 2.41s	remaining: 11.3s
176:	learn: 0.3244243	total: 2.42s	remaining: 11.3s
177:	learn: 0.3242889	total: 2.43s	remaining: 11.2s
178:	learn: 0.3241491	total: 2.44s	remaining: 11.2s
179:	learn: 0.3239570	total: 2.46s	remaining: 11.2s
180:	learn: 0.3238198	total: 2.47s	remaining: 11.2s
181:	learn: 0.3236172	total: 2.48s	remaining: 11.1s
182:	learn: 0.3235357	total: 2.49s	remaining: 11.1s
183:	learn: 0.3234009	total: 2.5s	remaining: 11.1s
184:	learn: 0.3232479	total: 2.52s	remaining: 11.1s
185:	learn: 0.3231872	total: 2.53s	remaining: 11.1s
186:	learn: 0.3230647	total: 2.54s	remaining: 11s
187:	learn: 0.3228525	total: 2.55s	remaining: 11s
188:	learn: 0.3227522	total: 2.57s	remaining: 11s
189:	learn: 0.3226435	total: 2.58s	remaining: 11s
190:	learn: 0.3224820	total: 2.59s	remaining: 11s
191:	learn: 0.3224304	total: 2.6s	remaining: 11s
192:	learn: 0.3222950	tota

337:	learn: 0.3060756	total: 4.42s	remaining: 8.66s
338:	learn: 0.3060040	total: 4.43s	remaining: 8.65s
339:	learn: 0.3059047	total: 4.45s	remaining: 8.63s
340:	learn: 0.3058299	total: 4.46s	remaining: 8.62s
341:	learn: 0.3058041	total: 4.47s	remaining: 8.6s
342:	learn: 0.3057032	total: 4.48s	remaining: 8.59s
343:	learn: 0.3055678	total: 4.5s	remaining: 8.57s
344:	learn: 0.3054548	total: 4.51s	remaining: 8.56s
345:	learn: 0.3053561	total: 4.52s	remaining: 8.55s
346:	learn: 0.3052362	total: 4.53s	remaining: 8.53s
347:	learn: 0.3051709	total: 4.55s	remaining: 8.52s
348:	learn: 0.3050340	total: 4.56s	remaining: 8.5s
349:	learn: 0.3049199	total: 4.57s	remaining: 8.49s
350:	learn: 0.3048618	total: 4.58s	remaining: 8.47s
351:	learn: 0.3047552	total: 4.6s	remaining: 8.46s
352:	learn: 0.3046429	total: 4.61s	remaining: 8.45s
353:	learn: 0.3045626	total: 4.62s	remaining: 8.44s
354:	learn: 0.3044763	total: 4.64s	remaining: 8.42s
355:	learn: 0.3043858	total: 4.65s	remaining: 8.41s
356:	learn: 0.30

508:	learn: 0.2937476	total: 6.7s	remaining: 6.46s
509:	learn: 0.2936959	total: 6.71s	remaining: 6.45s
510:	learn: 0.2936389	total: 6.72s	remaining: 6.43s
511:	learn: 0.2935569	total: 6.73s	remaining: 6.42s
512:	learn: 0.2935156	total: 6.75s	remaining: 6.4s
513:	learn: 0.2934046	total: 6.76s	remaining: 6.39s
514:	learn: 0.2933484	total: 6.77s	remaining: 6.38s
515:	learn: 0.2932913	total: 6.78s	remaining: 6.36s
516:	learn: 0.2932131	total: 6.79s	remaining: 6.35s
517:	learn: 0.2931906	total: 6.81s	remaining: 6.33s
518:	learn: 0.2931210	total: 6.82s	remaining: 6.32s
519:	learn: 0.2930944	total: 6.83s	remaining: 6.3s
520:	learn: 0.2930348	total: 6.84s	remaining: 6.29s
521:	learn: 0.2929644	total: 6.86s	remaining: 6.28s
522:	learn: 0.2928890	total: 6.87s	remaining: 6.26s
523:	learn: 0.2928359	total: 6.88s	remaining: 6.25s
524:	learn: 0.2927485	total: 6.89s	remaining: 6.24s
525:	learn: 0.2926998	total: 6.91s	remaining: 6.22s
526:	learn: 0.2926240	total: 6.92s	remaining: 6.21s
527:	learn: 0.2

682:	learn: 0.2838538	total: 9.16s	remaining: 4.25s
683:	learn: 0.2838430	total: 9.18s	remaining: 4.24s
684:	learn: 0.2838249	total: 9.19s	remaining: 4.22s
685:	learn: 0.2837576	total: 9.2s	remaining: 4.21s
686:	learn: 0.2836889	total: 9.22s	remaining: 4.2s
687:	learn: 0.2836603	total: 9.23s	remaining: 4.19s
688:	learn: 0.2836332	total: 9.25s	remaining: 4.17s
689:	learn: 0.2836133	total: 9.26s	remaining: 4.16s
690:	learn: 0.2835449	total: 9.28s	remaining: 4.15s
691:	learn: 0.2835032	total: 9.29s	remaining: 4.14s
692:	learn: 0.2834353	total: 9.31s	remaining: 4.12s
693:	learn: 0.2834205	total: 9.32s	remaining: 4.11s
694:	learn: 0.2833935	total: 9.34s	remaining: 4.1s
695:	learn: 0.2833504	total: 9.35s	remaining: 4.08s
696:	learn: 0.2832594	total: 9.37s	remaining: 4.07s
697:	learn: 0.2832424	total: 9.38s	remaining: 4.06s
698:	learn: 0.2831691	total: 9.39s	remaining: 4.04s
699:	learn: 0.2831590	total: 9.41s	remaining: 4.03s
700:	learn: 0.2830965	total: 9.42s	remaining: 4.02s
701:	learn: 0.2

...

186:	learn: 0.2871780	total: 1.78s	remaining: 7.76s
187:	learn: 0.2870229	total: 1.79s	remaining: 7.75s
188:	learn: 0.2869121	total: 1.8s	remaining: 7.73s
189:	learn: 0.2868171	total: 1.81s	remaining: 7.72s
190:	learn: 0.2867580	total: 1.82s	remaining: 7.7s
191:	learn: 0.2866367	total: 1.83s	remaining: 7.69s
192:	learn: 0.2864913	total: 1.83s	remaining: 7.67s
193:	learn: 0.2864024	total: 1.84s	remaining: 7.66s
194:	learn: 0.2863146	total: 1.85s	remaining: 7.65s
195:	learn: 0.2861683	total: 1.86s	remaining: 7.64s
196:	learn: 0.2860661	total: 1.87s	remaining: 7.63s
197:	learn: 0.2860128	total: 1.88s	remaining: 7.61s
198:	learn: 0.2859212	total: 1.89s	remaining: 7.6s
199:	learn: 0.2858409	total: 1.9s	remaining: 7.59s
200:	learn: 0.2857667	total: 1.91s	remaining: 7.58s
201:	learn: 0.2856289	total: 1.92s	remaining: 7.57s
202:	learn: 0.2855344	total: 1.92s	remaining: 7.55s
203:	learn: 0.2854045	total: 1.93s	remaining: 7.55s
204:	learn: 0.2853063	total: 1.94s	remaining: 7.54s
205:	learn: 0.28

347:	learn: 0.2714620	total: 3.4s	remaining: 6.36s
348:	learn: 0.2714019	total: 3.42s	remaining: 6.38s
349:	learn: 0.2713491	total: 3.44s	remaining: 6.39s
350:	learn: 0.2712639	total: 3.46s	remaining: 6.39s
351:	learn: 0.2712223	total: 3.47s	remaining: 6.39s
352:	learn: 0.2711775	total: 3.49s	remaining: 6.39s
353:	learn: 0.2711273	total: 3.5s	remaining: 6.39s
354:	learn: 0.2710705	total: 3.52s	remaining: 6.39s
355:	learn: 0.2709671	total: 3.53s	remaining: 6.39s
356:	learn: 0.2709035	total: 3.55s	remaining: 6.39s
357:	learn: 0.2708686	total: 3.56s	remaining: 6.38s
358:	learn: 0.2707884	total: 3.57s	remaining: 6.37s
359:	learn: 0.2707223	total: 3.58s	remaining: 6.37s
360:	learn: 0.2706427	total: 3.59s	remaining: 6.36s
361:	learn: 0.2705865	total: 3.6s	remaining: 6.35s
362:	learn: 0.2705341	total: 3.62s	remaining: 6.35s
363:	learn: 0.2704560	total: 3.63s	remaining: 6.34s
364:	learn: 0.2703923	total: 3.64s	remaining: 6.34s
365:	learn: 0.2703261	total: 3.65s	remaining: 6.33s
366:	learn: 0.2

517:	learn: 0.2613728	total: 5.22s	remaining: 4.86s
518:	learn: 0.2613417	total: 5.23s	remaining: 4.85s
519:	learn: 0.2613136	total: 5.24s	remaining: 4.84s
520:	learn: 0.2612927	total: 5.25s	remaining: 4.83s
521:	learn: 0.2612474	total: 5.26s	remaining: 4.82s
522:	learn: 0.2611978	total: 5.27s	remaining: 4.81s
523:	learn: 0.2611546	total: 5.28s	remaining: 4.8s
524:	learn: 0.2611136	total: 5.29s	remaining: 4.79s
525:	learn: 0.2610372	total: 5.31s	remaining: 4.78s
526:	learn: 0.2610147	total: 5.32s	remaining: 4.77s
527:	learn: 0.2609814	total: 5.33s	remaining: 4.76s
528:	learn: 0.2609338	total: 5.34s	remaining: 4.75s
529:	learn: 0.2608930	total: 5.35s	remaining: 4.74s
530:	learn: 0.2608766	total: 5.36s	remaining: 4.73s
531:	learn: 0.2608218	total: 5.37s	remaining: 4.72s
532:	learn: 0.2607607	total: 5.37s	remaining: 4.71s
533:	learn: 0.2607087	total: 5.38s	remaining: 4.7s
534:	learn: 0.2606685	total: 5.39s	remaining: 4.69s
535:	learn: 0.2606342	total: 5.41s	remaining: 4.68s
536:	learn: 0.

685:	learn: 0.2540987	total: 6.86s	remaining: 3.14s
686:	learn: 0.2540561	total: 6.88s	remaining: 3.13s
687:	learn: 0.2540023	total: 6.88s	remaining: 3.12s
688:	learn: 0.2539632	total: 6.89s	remaining: 3.11s
689:	learn: 0.2539339	total: 6.9s	remaining: 3.1s
690:	learn: 0.2539004	total: 6.91s	remaining: 3.09s
691:	learn: 0.2538856	total: 6.92s	remaining: 3.08s
692:	learn: 0.2538478	total: 6.93s	remaining: 3.07s
693:	learn: 0.2538047	total: 6.94s	remaining: 3.06s
694:	learn: 0.2537755	total: 6.95s	remaining: 3.05s
695:	learn: 0.2537521	total: 6.96s	remaining: 3.04s
696:	learn: 0.2537007	total: 6.97s	remaining: 3.03s
697:	learn: 0.2536801	total: 6.98s	remaining: 3.02s
698:	learn: 0.2536625	total: 6.99s	remaining: 3.01s
699:	learn: 0.2536421	total: 7s	remaining: 3s
700:	learn: 0.2535871	total: 7s	remaining: 2.99s
701:	learn: 0.2535507	total: 7.01s	remaining: 2.98s
702:	learn: 0.2534943	total: 7.02s	remaining: 2.97s
703:	learn: 0.2534339	total: 7.04s	remaining: 2.96s
704:	learn: 0.2533989	t

859:	learn: 0.2476279	total: 8.47s	remaining: 1.38s
860:	learn: 0.2475702	total: 8.48s	remaining: 1.37s
861:	learn: 0.2475376	total: 8.49s	remaining: 1.36s
862:	learn: 0.2474991	total: 8.5s	remaining: 1.35s
863:	learn: 0.2474350	total: 8.51s	remaining: 1.34s
864:	learn: 0.2473849	total: 8.52s	remaining: 1.33s
865:	learn: 0.2473280	total: 8.53s	remaining: 1.32s
866:	learn: 0.2472949	total: 8.53s	remaining: 1.31s
867:	learn: 0.2472247	total: 8.54s	remaining: 1.3s
868:	learn: 0.2472118	total: 8.55s	remaining: 1.29s
869:	learn: 0.2471628	total: 8.56s	remaining: 1.28s
870:	learn: 0.2471412	total: 8.57s	remaining: 1.27s
871:	learn: 0.2471216	total: 8.58s	remaining: 1.26s
872:	learn: 0.2470714	total: 8.59s	remaining: 1.25s
873:	learn: 0.2470371	total: 8.6s	remaining: 1.24s
874:	learn: 0.2470233	total: 8.6s	remaining: 1.23s
875:	learn: 0.2469933	total: 8.61s	remaining: 1.22s
876:	learn: 0.2469764	total: 8.62s	remaining: 1.21s
877:	learn: 0.2469445	total: 8.63s	remaining: 1.2s
878:	learn: 0.246